## 빅데이터 실습

### 제주도 핫플레이스 시각화

#### 데이터 불러오기

In [67]:
# 필요 라이브러리 사용등록
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster    # 마커클러스터용 클래스 추가

In [2]:
# 추가 라이브러리 사용등록
import requests
import time
from tqdm import tqdm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager,rc

font_path = 'C:/windows/Fonts/NanumGothicCoding.ttf'#나눔고딕체 사용
font = font_manager.FontProperties(fname=font_path).get_name() #실제 설치된 폰트 이름조회
rc('font', family=font) #한글깨짐현상 해결!!
rcParams['axes.unicode_minus'] = False #한글 사용시 마이너스 표시 깨짐 해결!

In [5]:
## 데이터 불러오기
rawTotal = pd.read_excel('../day04/data/1_crawling_raw.xlsx')
rawTotal.tail()

,content,date,like,place,tags
8796,Now it's purple😆💜수색 넘 이쀼리 🤭 올해는 아퓨지말구 즐거운 일만 가...,2019-01-01,42,NaN,"['#일상', '#구기자', '#티타임', '#2019', '#새해', '#차스타그..."
8797,.간다간다간다#제주 #친정여행 #겨울방학 #제주여행앞머리는 괜히 전날 잘라서 또순이...,2019-01-01,50,NaN,"['#제주', '#친정여행', '#겨울방학', '#제주여행앞머리는', '#친정찬스'..."
8798,2019년 모두 복 많이많이 받고 풍요로운 한해 보내길 ♥️#사진좀찍으시는님덕에건져...,2019-01-01,68,Jeju,"['#사진좀찍으시는님덕에건져또', '#이순간']"
8799,2018.12.25 ⠀룸에 의자가 두개 밖에 없어서 ⠀이현이는 유모차에 앉아서 식사...,2019-01-01,70,NaN,[]
8800,.2019년 첫날디스이즈핫에서 새해를 맞이하시는 손님들께 떡국을 준비했습니다.한 해...,2019-01-01,48,디스이즈핫,['#HappyNewYearEveryone']


In [6]:
# 위치정보 확인
locCounts = rawTotal['place'].value_counts()

In [7]:
locCounts.head()

place
Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Seogwipo                 66
제주도 크리스마스 박물관            59
Name: count, dtype: int64

In [8]:
# DF로 변경
dfLocCounts = pd.DataFrame(locCounts)
dfLocCounts.head(3)

,count
place,
Jeju,271
Jeju-do,179
Jungle Book by Alice,108


In [9]:
dfLocCounts.to_excel('./data/3_location_counts.xlsx')

In [10]:
Locations = list(dfLocCounts.index)

In [11]:
Locations

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '할로비치',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '고집돌우럭중문점',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 '제주신화월드 Jeju Shinhwa World',
 'Woljeongri Beach',
 '협재해변 Beach',
 '연동 바오젠 거리',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '고래배꼽',
 '제주 성산 신산포구자연산횟집 회포장',
 '새별오름',
 '용머리해안',
 '용눈이오름',
 '캔디원',
 '사려니숲길',
 '하이엔드 제주',
 '제주빅볼랜드',
 '서귀포 느영나영 게스트하우스',
 '휴애리 자연생활공원',
 '옹포리',
 '삼무공원',
 '마마뜰',
 '제주어린왕자게스트하우스',
 '위미동백나무군락',
 '제주커피박물관 Baum',
 'Jeju Aewol',
 '한라산',
 '위미2리 동백군락지',
 'Jeju Island 제주특별자치도 济州道',
 '광치기해변',
 '서우봉',
 '제주해남',
 '한라산 (漢拏山, Hallasan)',
 '카페한라산',
 '성이시돌목장',
 '제주 송악산',
 '옹포별장가든',
 '월정리카페콧수염',
 '牛岛 Udo Island 우도',
 '곽지해수욕장',
 '카페브리프',
 '아줄레주',
 '에코랜드',
 '산굼부리',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 'Terarosa - 테라로사',

##### 카카오 OPEN API로 장소 검색
- kakao developers 사이트에서 애플리케이션 등록
- key를 생성


In [31]:
shopName = '헬로비치'

# keyword는 가게명, address는 가게 주소로 검색
url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'

header = { 'Authorization' : 'KakaoAK 본인키'} # 키는 본인키 사용

places = requests.get(url, headers=header).json()['documents']
places

[{'address_name': '강원특별자치도 동해시 묵호진동 2-280',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 한식 > 해물,생선 > 게,대게',
  'distance': '',
  'id': '1910026193',
  'phone': '',
  'place_name': '헬로비치 본점',
  'place_url': 'http://place.map.kakao.com/1910026193',
  'road_address_name': '강원특별자치도 동해시 일출로 103',
  'x': '129.11873488492282',
  'y': '37.55246010486625'},
 {'address_name': '서울 성북구 안암동4가 41-39',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '서비스,산업 > 제조업 > 섬유 > 직물,원단',
  'distance': '',
  'id': '14102647',
  'phone': '070-8795-6545',
  'place_name': '헬로키티비치타올',
  'place_url': 'http://place.map.kakao.com/14102647',
  'road_address_name': '서울 성북구 안암로5길 8',
  'x': '127.024308430319',
  'y': '37.5795979359826'}]

In [23]:
## 장소 검색 함수 findPlace
def findPlace(shopName):
    # 1. 검색API 주소변수
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'
    # 2. header 구성
    header = { 'Authorization' : 'KakaoAK 본인키'} # 키는 본인키 사용
    # 3. API를 웹으로 요청, 결과 return
    places = requests.get(url, headers=header).json()['documents']
    # 4. 필요정보만 추출
    place = places[0]
    realName = place['place_name']
    lat = place['y']    # 위도(latitude)
    lng = place['x']    # 경도(longitude)
    phoneNum = place['phone']
    roadAddress = place['address_name']

    return [shopName, realName, phoneNum, roadAddress, lat, lng]

In [24]:
## 테스트
result = findPlace('1100고지')
result

['1100고지',
 '한라산1100고지',
 '',
 '제주특별자치도 서귀포시 색달동 산 1-2',
 '33.3580781709788',
 '126.462219691112']

In [25]:
# Locations 리스트를 반복문으로 주소가져오는 처리
locationResults = []

for loc in tqdm(Locations):
    try:
        data = findPlace(loc)
        locationResults.append(data)
        time.sleep(0.5)
    except:
        pass

100%|██████████| 1028/1028 [13:25<00:00,  1.28it/s]


In [26]:
locationResults

[['Jeju',
  'R고기 in Jeju',
  '064-744-6222',
  '제주특별자치도 제주시 도두일동 2623-4',
  '33.503587560005954',
  '126.46542934219148'],
 ['Seogwipo',
  '서귀포잠수함',
  '064-732-6060',
  '제주특별자치도 서귀포시 서홍동 707-5',
  '33.2393033784206',
  '126.558616052674'],
 ['제주도 크리스마스 박물관',
  '바이나흐튼 크리스마스박물관',
  '010-4602-7976',
  '제주특별자치도 서귀포시 안덕면 서광리 456',
  '33.29131209935493',
  '126.32779866033343'],
 ['할로비치',
  '할로비치',
  '',
  '서울 강남구 신사동 511-7',
  '37.5182108620338',
  '127.020825988861'],
 ['제주에인감귤밭',
  '제주에인감귤밭',
  '',
  '제주특별자치도 서귀포시 호근동 693',
  '33.25656069063887',
  '126.5390139270361'],
 ['Jeju Island',
  '제주도',
  '',
  '제주특별자치도 제주시 오등동 산 182',
  '33.379777816446165',
  '126.54587355630036'],
 ['1100고지',
  '한라산1100고지',
  '',
  '제주특별자치도 서귀포시 색달동 산 1-2',
  '33.3580781709788',
  '126.462219691112'],
 ['월정리해변',
  '월정리해수욕장',
  '064-728-3394',
  '제주특별자치도 제주시 구좌읍 월정리 33-3',
  '33.556469394054',
  '126.795805057888'],
 ['고집돌우럭중문점',
  '고집돌우럭 중문점',
  '0507-1408-1540',
  '제주특별자치도 서귀포시 색달동 2351',
  '33.2579811121134'

In [27]:
## list --> DataFrame 변경
dfLocCountResults = pd.DataFrame(locationResults)

In [29]:
dfLocCountResults.columns = ['초기이름', '실제이름', '전화번호', '주소', '위도', '경도']

In [30]:
dfLocCountResults.to_excel('./data/3_locations.xlsx', index=False)

### 인스타그램 크롤링정보와 카카오API로 주소 검색결과정보 병합

In [32]:
dfLocCounts.head()

,count
place,
Jeju,271
Jeju-do,179
Jungle Book by Alice,108
Seogwipo,66
제주도 크리스마스 박물관,59


In [33]:
dfLocCountResults.head()

,초기이름,실제이름,전화번호,주소,위도,경도
0,Jeju,R고기 in Jeju,064-744-6222,제주특별자치도 제주시 도두일동 2623-4,33.503587560005954,126.46542934219148
1,Seogwipo,서귀포잠수함,064-732-6060,제주특별자치도 서귀포시 서홍동 707-5,33.2393033784206,126.558616052674
2,제주도 크리스마스 박물관,바이나흐튼 크리스마스박물관,010-4602-7976,제주특별자치도 서귀포시 안덕면 서광리 456,33.29131209935493,126.32779866033343
3,할로비치,할로비치,,서울 강남구 신사동 511-7,37.5182108620338,127.020825988861
4,제주에인감귤밭,제주에인감귤밭,,제주특별자치도 서귀포시 호근동 693,33.25656069063887,126.5390139270361


In [35]:
# reset_index로 place를 일반컬럼으로 변경
dfLocCounts.reset_index(inplace=True)

In [36]:
dfLocCounts.head()

,place,count
0,Jeju,271
1,Jeju-do,179
2,Jungle Book by Alice,108
3,Seogwipo,66
4,제주도 크리스마스 박물관,59


In [39]:
locationData = pd.merge(left=dfLocCountResults, right=dfLocCounts, how='inner', left_on='초기이름', right_on='place')

In [40]:
locationData.tail()

,초기이름,실제이름,전화번호,주소,위도,경도,place,count
769,제주마당,제주마당,064-749-5501,제주특별자치도 제주시 이호일동 663-1,33.49810414027896,126.45817446683485,제주마당,1
770,형제섬,형제섬,,경남 거제시 남부면,34.7109149877063,128.644665989846,형제섬,1
771,동복해녀식당,동복해녀식당,064-783-4141,제주특별자치도 제주시 구좌읍 동복리 1647,33.553472573104635,126.70694723770639,동복해녀식당,1
772,종로 광장시장,광장시장,02-2267-0291,서울 종로구 예지동 2-1,37.570033672535104,126.9989416227693,종로 광장시장,1
773,디스이즈핫,디스이즈핫,,제주특별자치도 제주시 구좌읍 하도리 59-1,33.5131489720297,126.897326953126,디스이즈핫,1


In [42]:
condition = locationData['실제이름'] == '제주동문시장'

In [43]:
locationData[condition]

,초기이름,실제이름,전화번호,주소,위도,경도,place,count
71,제주동문재래시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주동문재래시장,5
211,제주시 동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주시 동문시장,2
296,제주시동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,제주시동문시장,1
461,동문시장,제주동문시장,,제주특별자치도 제주시 일도일동 1148-2,33.51278396493313,126.52830338722843,동문시장,1


In [44]:
## 중복데이터 확인
locationData['실제이름'].value_counts()

실제이름
제주동문시장                4
올레길 7코스(서귀포-월평 올레)    4
세화해변                  3
에코랜드테마파크              3
오설록 티뮤지엄              3
                     ..
수제홍과일모찌 애월하귀점         1
대천해수욕장                1
도라지식당                 1
호박다방                  1
디스이즈핫                 1
Name: count, Length: 708, dtype: int64

In [46]:
## 중복데이터 pivot
locationDataFinal = locationData.pivot_table(values='count', index=['실제이름', '주소', '위도', '경도'], aggfunc='sum')

In [49]:
locationDataFinal.reset_index(inplace=True)

In [51]:
locationDataFinal.tail()

,실제이름,주소,위도,경도,count
704,휴애리자연생활공원,제주특별자치도 서귀포시 남원읍 신례리 2081,33.309410458656,126.635409752107,11
705,흑돼지가있는풍경 본점,제주특별자치도 제주시 노형동 668,33.475375671553,126.480646164645,1
706,흑송,부산 해운대구 송정동 443-3,35.1750106256909,129.196512928977,1
707,히든클리프호텔&네이쳐,제주특별자치도 서귀포시 상예동 625,33.2547764885257,126.402595131766,9
708,히비,서울 용산구 원효로1가 128-12,37.5384855299872,126.968454416507,1


In [52]:
locationDataFinal.to_excel('./data/3_locations_final.xlsx')

In [53]:
locationDataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   실제이름    709 non-null    object
 1   주소      709 non-null    object
 2   위도      709 non-null    object
 3   경도      709 non-null    object
 4   count   709 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 27.8+ KB


In [56]:
## 위도, 경도가 object에서 float64로 변경
locationDataFinal = locationDataFinal.astype({'위도':'float64', '경도':'float64'})

In [58]:
locationDataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   실제이름    709 non-null    object 
 1   주소      709 non-null    object 
 2   위도      709 non-null    float64
 3   경도      709 non-null    float64
 4   count   709 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 27.8+ KB


##### 지도 시각화
- folium 사용

In [68]:
locHanlaMt = [33.362050, 126.530087]
map = folium.Map(location=locHanlaMt, zoom_start=11, tiles='OpenStreetMap')

for i in range(len(locationDataFinal)):
    name = locationDataFinal['실제이름'][i] # DF 컬럼이름 행값 == i
    count = int(locationDataFinal['count'][i])
    lat = float(locationDataFinal['위도'][i])
    lng = float(locationDataFinal['경도'][i])
    folium.CircleMarker(location=[lat, lng], radius=count, color='red', popup=name).add_to(map)
map



In [66]:
map.save('./data/3_jeju_hotplaces.html')

In [75]:
## 마커 클러스터
mapLocs = []
mapNames = []

for i in range(len(locationDataFinal)):
    data = locationDataFinal.iloc[i]
    mapLocs.append((float(data['위도']), float(data['경도'])))
    mapNames.append(data['실제이름'])
    
locHanlaMt = [33.362050, 126.530087]
map = folium.Map(location=locHanlaMt, zoom_start=11)

markerCluster = MarkerCluster(locations=mapLocs, popups=mapNames, name='제주 핫플레이스', overlay=True, control=True)
markerCluster.add_to(map)
folium.LayerControl().add_to(map)

map

In [76]:
map.save('./data/3_jeju_hotplaces_cluster.html')